# Estudos de caso: Andes, América do Sul

Notebook que demonstrará alguns estudos de caso e a relação do distúrbio de gravidade e da anomalia bouguer na interpretação de dados de gravidade em margem convergente.

# Importando bibliotecas

Importando as bibliotecas que serão utilizadas para manipulação e visualização dos dados de gravidade.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import harmonica as hm
import cartopy.crs as ccrs
import matplotlib.colors as colors
import boule as bl
import pyproj
import verde as vd

# Carregando os dados

In [ ]:
fname = 'dataset/gravity_earth_andes.gdf'
gravity = hm.load_icgem_gdf(fname)

In [ ]:
fname = 'dataset/geoid_andes.gdf'
geoid = hm.load_icgem_gdf(fname)

In [ ]:
fname = 'dataset/topography_andes.gdf'
topo = hm.load_icgem_gdf(fname)

### Armazenando os dados em um array e manipulando os dados

In [ ]:
lat = gravity.latitude
lon = gravity.longitude
elevation = gravity.h_over_geoid
data = gravity.gravity_earth
geoidal_height = geoid.geoid
topography = topo.topography_shm

In [ ]:
longitude, latitude = np.meshgrid(lon.values,lat.values)

In [ ]:
field = data.values.reshape(longitude.shape)

# Gravidade real 

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Gravity (Andes)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([lon.values.min(),lon.values.max(),
               lat.values.min(),lat.values.max()], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(longitude,latitude,field,
                   100,transform=ccrs.PlateCarree(),
                   cmap='viridis')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('mGal',fontsize=10)

plt.savefig('images/gravity_earth_andes.png',dpi=200,bbox_inches='tight')

plt.show()

# Gravidade Normal sobre a superfície terrestre

In [ ]:
normal_gravity = bl.WGS84.normal_gravity(latitude, elevation)

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Normal gravity (Andes)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([lon.values.min(),lon.values.max(),
               lat.values.min(),lat.values.max()], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(longitude,latitude,normal_gravity,
                   100,transform=ccrs.PlateCarree(),
                   cmap='viridis')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('mGal',fontsize=10)

plt.savefig('images/normal_gravity_andes.png',dpi=200,bbox_inches='tight')

plt.show()

# Distúrbio de gravidade 

In [ ]:
disturbance = field - normal_gravity

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Gravity Disturbance (Andes)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([lon.values.min(),lon.values.max(),
               lat.values.min(),lat.values.max()], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(lon,lat,disturbance,
                   100,transform=ccrs.PlateCarree(),
                   norm=colors.CenteredNorm(),
                   cmap='RdBu_r')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('mGal',fontsize=10)

plt.savefig('images/disturbance_andes.png',dpi=200,bbox_inches='tight')

plt.show()

# Topografia

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Topography (Andes)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([lon.values.min(),lon.values.max(),
               lat.values.min(),lat.values.max()], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(longitude,latitude,topography,
                   100,transform=ccrs.PlateCarree(),
                   norm=colors.CenteredNorm(),
                   cmap='terrain')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('meters',fontsize=10)

plt.savefig('images/topography_andes.png',dpi=200,bbox_inches='tight')

plt.show()

# Anomalia Bouguer

In [ ]:
reference = geoidal_height + topography
bouguer_correction = hm.bouguer_correction(reference)
bouguer_anomaly = disturbance - bouguer_correction

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Bouguer Anomaly (Andes)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([lon.values.min(),lon.values.max(),
               lat.values.min(),lat.values.max()], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(longitude,latitude,bouguer_anomaly,
                   100,transform=ccrs.PlateCarree(),
                   norm=colors.CenteredNorm(),
                   cmap='RdBu_r')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('mGal',fontsize=10)

plt.savefig('images/bouguer_andes.png',dpi=200,bbox_inches='tight')

plt.show()

# Extraindo e visualizando os dados em um perfil

## Projeção do sistema de coordenadas geográfico para o sistema de coordenadas cartesiano

In [ ]:
projection = pyproj.Proj(proj="merc", lat_ts=lat.values.mean())

In [ ]:
proj_coords = projection(longitude,latitude)

## Definindo a redução dos dados

In [ ]:
spacing = 15/60
reducer = vd.BlockReduce(reduction="mean", spacing=spacing * 111*1e3)

## Perfil do Distúrbio de gravidade

In [ ]:
filter_coords, filter_disturbance = reducer.filter(proj_coords, disturbance)
spline = vd.Spline(damping=1e-10).fit(filter_coords, filter_disturbance)

In [ ]:
point1 = (-75.5,-22.0)
point2 = (-62.5,-22.0)
line_disturbance = spline.profile(
    point1=point1,
    point2=point2,
    size=300,
    projection=projection,
    dims=("latitude","longitude"),
    data_names=("disturbance"),
)

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Distúrbio de gravidade (Andes)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([lon.values.min(),lon.values.max(),
               lat.values.min(),lat.values.max()], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(longitude,latitude,disturbance,
                   100,transform=ccrs.PlateCarree(),
                   norm=colors.CenteredNorm(),
                   cmap='RdBu_r')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('mGal',fontsize=10)

ax.plot(line_disturbance.longitude, line_disturbance.latitude, "-k", transform=ccrs.PlateCarree())
ax.text(point1[0], point1[1], "A", transform=ccrs.PlateCarree())
ax.text(point2[0], point2[1], "B", transform=ccrs.PlateCarree())

plt.savefig('images/disturbance_profile_andes.png',dpi=200,bbox_inches='tight')

plt.show()

## Perfil da Anomalia Bouguer

In [ ]:
filter_coords, filter_bouguer = reducer.filter(proj_coords, bouguer_anomaly)
spline = vd.Spline(damping=1e-10).fit(filter_coords, filter_bouguer)

In [ ]:
point1 = (-75.5,-22.0)
point2 = (-62.5,-22.0)
line_bouguer = spline.profile(
    point1=point1,
    point2=point2,
    size=300,
    projection=projection,
    dims=("latitude","longitude"),
    data_names=("bouguer_anomaly"),
)

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Anomalia Bouguer (Andes)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([lon.values.min(),lon.values.max(),
               lat.values.min(),lat.values.max()], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(longitude,latitude,bouguer_anomaly,
                   100,transform=ccrs.PlateCarree(),
                   norm=colors.CenteredNorm(),
                   cmap='RdBu_r')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('mGal',fontsize=10)

ax.plot(line_bouguer.longitude, line_bouguer.latitude, "-k", transform=ccrs.PlateCarree())
ax.text(point1[0], point1[1], "A", transform=ccrs.PlateCarree())
ax.text(point2[0], point2[1], "B", transform=ccrs.PlateCarree())

plt.savefig('images/bouguer_profile_andes.png',dpi=200,bbox_inches='tight')

plt.show()

## Perfil de Topografia

In [ ]:
filter_coords, filter_topo = reducer.filter(proj_coords, topography)
spline = vd.Spline(damping=1e-10).fit(filter_coords, filter_topo)

In [ ]:
point1 = (-75.5,-22.0)
point2 = (-62.5,-22.0)
line_topo = spline.profile(
    point1=point1,
    point2=point2,
    size=300,
    projection=projection,
    dims=("latitude","longitude"),
    data_names=("topography"),
)

In [ ]:
fig = plt.figure(figsize=[6,6])

ax = fig.add_subplot(projection=ccrs.PlateCarree())
ax.set_title('Topografia (Andes)',fontsize=15)
ax.coastlines(linewidth=0.5)
ax.set_extent([lon.values.min(),lon.values.max(),
               lat.values.min(),lat.values.max()], ccrs.PlateCarree())
gd = ax.gridlines(visible=None,draw_labels=True, dms=False, x_inline=False, y_inline=False)
gd.top_labels = False
gd.right_labels = False

cmap = ax.contourf(longitude,latitude,topography,
                   100,transform=ccrs.PlateCarree(),
                   norm=colors.CenteredNorm(),
                   cmap='terrain')

cbar = fig.colorbar(cmap,orientation='vertical',pad=0.02,aspect=50,spacing='uniform',ax=ax,fraction=0.011)
cbar.set_label('meters',fontsize=10)

ax.plot(line_topo.longitude, line_topo.latitude, "-k", transform=ccrs.PlateCarree())
ax.text(point1[0], point1[1], "A", transform=ccrs.PlateCarree())
ax.text(point2[0], point2[1], "B", transform=ccrs.PlateCarree())

plt.savefig('images/topography_profile_andes.png',dpi=200,bbox_inches='tight')

plt.show()

### Visualizando tudo em um perfil

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True,figsize=(10, 5))

ax1.set_title("Profile (A-B)")
ax1.plot(line_disturbance.distance,line_disturbance.disturbance,'-b',label='Disturbio')
ax1.plot(line_bouguer.distance,line_bouguer.bouguer_anomaly,'-r',label='Bouguer')
ax1.set_ylabel("Gravity (m)")
ax1.set_xlim(line_disturbance.distance.min(), line_disturbance.distance.max())
ax1.grid()

ax2.fill_between([line_topo.distance.min(), line_topo.distance.max()], [0, 0], -8000, color='blue')
ax2.fill_between(line_topo.distance,line_topo.topography,-8000.,color='black')
ax2.set_xlabel("Distance (m)")
ax2.set_ylabel("Topography (m)")
ax2.set_xlim(line_topo.distance.min(), line_topo.distance.max())
ax2.set_ylim(line_topo.topography.min(), line_topo.topography.max() + 100.)

plt.savefig('images/profile_andes.png',dpi=200,bbox_inches='tight')

plt.show()